# Sparkathon implementation


In [46]:
%pip install groq chromadb sentence-transformers gradio requests sarvamai pydub soundfile

In [47]:
import json
import chromadb
from sentence_transformers import SentenceTransformer

# Load SentenceTransformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Load product JSON
with open("dataset.json", "r") as f:
    products = json.load(f)

# ✅ Initialize persistent ChromaDB client
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="products")

def clean_metadata(metadata: dict) -> dict:
    return {
        k: v for k, v in metadata.items()
        if k != "reviews" and isinstance(v, (str, int, float, bool))
    }

# Index documents
for idx, item in enumerate(products):
    doc = f"""
    Product: {item['product_name']}
    Price: {item['product_price']}
    Details: {item['product_details']}
    Specs: {item['specifications']}
    Rating: {item['rating']}
    Availability: {item['availability']}
    """
    embedding = model.encode(doc).tolist()

    collection.add(
        ids=[str(idx)],
        embeddings=[embedding],
        documents=[doc],
        metadatas=[clean_metadata(item)]
    )


In [48]:
# Fetch all items in the collection (assuming you used 'products' as the name)
collection = client.get_collection(name="products")

# Query with a dummy vector to retrieve all documents
all_items = collection.get()

# Display each item nicely
for i in range(len(all_items['ids'])):
    print(f"🛒 Product ID: {all_items['ids'][i]}")
    print(f"📦 Document:\n{all_items['documents'][i]}")
    print(f"📌 Metadata:\n{all_items['metadatas'][i]}")
    print("-" * 80)


🛒 Product ID: 0
📦 Document:

    Product: Fresh Banana, Each
    Price: $0.29
    Details: Fresh, ripe bananas sold individually.
    Specs: Approximate weight: 0.3-0.5 lbs
    Rating: 4.5
    Availability: In stock
    
📌 Metadata:
{'product_details': 'Fresh, ripe bananas sold individually.', 'rating': 4.5, 'image_url': 'https://i5.walmartimages.com/asr/12345678-1234-5678-1234-567812345678.jpg', 'availability': 'In stock', 'product_price': '$0.29', 'product_name': 'Fresh Banana, Each', 'specifications': 'Approximate weight: 0.3-0.5 lbs'}
--------------------------------------------------------------------------------
🛒 Product ID: 1
📦 Document:

    Product: Fresh Strawberries, 1 lb
    Price: $3.98
    Details: Sweet and juicy strawberries, perfect for snacking or desserts.
    Specs: Weight: 1 lb
    Rating: 4.2
    Availability: In stock
    
📌 Metadata:
{'specifications': 'Weight: 1 lb', 'product_name': 'Fresh Strawberries, 1 lb', 'product_details': 'Sweet and juicy strawberries, 

In [49]:
import os

os.environ["GROQ_API_KEY"] = ""

os.environ["SARVAM_API_KEY"] = ""


In [50]:
from groq import Groq
from sentence_transformers import SentenceTransformer

# Load your SentenceTransformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Set up Groq client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))
groq_model = "llama3-70b-8192"


In [51]:
def rag_query_groq(user_query, n_results=5):
    # Step 1: Embed query
    query_embedding = model.encode(user_query).tolist()

    # Step 2: Query ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )

    # Step 3: Build context
    context = "\n\n".join(results["documents"][0])

    # Step 4: Format prompt
    prompt = f"""You are a helpful AI shopping assistant.
Use the following product descriptions to answer the user's question.

Product Descriptions:
{context}

User Question: {user_query}
Answer:"""

    # Step 5: Call Groq API
    completion = client.chat.completions.create(
        model=groq_model,
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant that answers product-related queries."},
            {"role": "user", "content": prompt}
        ]
    )

    return completion.choices[0].message.content.strip()


In [52]:
print(rag_query_groq("Suggest a juicy fruit under $2"))
print(rag_query_groq("Which dairy products are under $5 and high rated?"))

I'd be happy to help!

Based on your request, I recommend the Fresh Banana, Each, priced at $0.29. It's a juicy and ripe banana, perfect for snacking or adding to your favorite smoothie. Plus, it's an amazing deal at under $2!
Based on the product descriptions, the dairy products that are under $5 and have a high rating (4.0 or higher) are:

1. Great Value Skim Milk, Gallon, 128 fl oz - Price: $3.78, Rating: 4.2
2. Great Value 2% Reduced Fat Milk, Gallon, 128 fl oz - Price: $3.88, Rating: 4.2
3. Great Value Whole Vitamin D Milk, Gallon, 128 fl oz - Price: $3.98, Rating: 4.4

These three products meet the criteria of being under $5 and having a high rating.


In [53]:
from datetime import datetime
import json
import requests
import os
import base64
import io
from sarvamai import SarvamAI as SarvamClient

class ShoppingMemory:
    """Advanced memory system to track user's shopping journey"""
    def __init__(self):
        self.cart = []
        self.conversation_history = []
        self.session_start = datetime.now()
        self.current_step = "browsing"  # browsing, selecting, checkout, completed
        self.user_preferences = {}

    def add_to_cart(self, product_info):
        """Add product to cart"""
        existing_item = None
        for item in self.cart:
            if item["product"]["product_name"] == product_info["product_name"]:
                existing_item = item
                break
        if existing_item:
            existing_item["quantity"] += 1
        else:
            self.cart.append({
                "product": product_info,
                "added_at": datetime.now(),
                "quantity": 1
            })

    def remove_from_cart(self, product_name):
        self.cart = [item for item in self.cart if item["product"]["product_name"] != product_name]

    def get_cart_summary(self):
        if not self.cart:
            return "🛒 Your cart is empty."
        total = 0
        summary = "🛒 **Your Cart:**\n\n"
        for i, item in enumerate(self.cart, 1):
            price = float(item["product"]["product_price"].replace("$", ""))
            item_total = price * item["quantity"]
            total += item_total
            summary += f"{i}. **{item['product']['product_name']}**\n"
            summary += f"   • Price: {item['product']['product_price']} x {item['quantity']} = ${item_total:.2f}\n"
            summary += f"   • Rating: {item['product'].get('rating', 'N/A')}\n\n"
        summary += f"💰 **Total: ${total:.2f}**\n"
        summary += f"📦 **Items: {len(self.cart)}**"
        return summary

    def update_step(self, step):
        self.current_step = step

    def add_conversation(self, user_input, ai_response):
        entry = {
            "timestamp": datetime.now(),
            "user": user_input,
            "ai": ai_response,
            "step": self.current_step
        }
        self.conversation_history.append(entry)
        if len(self.conversation_history) > 20:
            self.conversation_history = self.conversation_history[-20:]

    def get_conversation_context(self, last_n=3):
        recent = self.conversation_history[-last_n:] if self.conversation_history else []
        context = ""
        for conv in recent:
            context += f"User: {conv['user']}\nAI: {conv['ai']}\n\n"
        return context

# Initialize shopping memory
shopping_memory = ShoppingMemory()

# Enhanced Sarvam AI integration using official SDK
class SarvamAI:
    """
    Official Sarvam AI SDK integration for:
    - Text-to-Speech (TTS)
    - Speech-to-Text (STT)
    - Translation
    """

    def __init__(self, api_key):
        self.client = SarvamClient(api_subscription_key=api_key)
        self.api_key = api_key

    def text_to_speech(self, text, speaker="Anushka", language="hi-IN"):
        """
        Convert text to speech using Sarvam TTS API
        """
        try:
            response = self.client.text_to_speech.convert(
                text=text,
                target_language_code=language,
                speaker=speaker,
                pitch=0.0,
                pace=1.0,
                loudness=1.0,
                speech_sample_rate=22050,
                enable_preprocessing=True,
                model="bulbul:v2"
            )

            if response and hasattr(response, 'audios') and response.audios:
                # Save audio file
                audio_data = base64.b64decode(response.audios[0])
                output_file = f"tts_output_{datetime.now().strftime('%Y%m%d_%H%M%S')}.wav"
                with open(output_file, "wb") as f:
                    f.write(audio_data)
                print(f"🔊 TTS: Audio saved as '{output_file}'")
                return output_file
            else:
                print("❌ TTS: No audio data received")
                return None

        except Exception as e:
            print(f"❌ TTS Error: {str(e)}")
            return None

    def translate_text(self, text, source_language="en-IN", target_language="hi-IN"):
        """
        Translate text using Sarvam Translation API
        """
        try:
            response = self.client.text.translate(
                input=text,
                source_language_code=source_language,
                target_language_code=target_language,
                speaker_gender="Male",
                mode="formal",
                model="mayura:v1",
                enable_preprocessing=True
            )

            if response and hasattr(response, 'translated_text'):
                return response.translated_text
            else:
                print("❌ Translation: No translated text received")
                return text

        except Exception as e:
            print(f"❌ Translation Error: {str(e)}")
            return text

    def speech_to_text(self, audio_file_path, language="hi-IN"):
        """
        Convert speech to text using Sarvam STT API
        """
        try:
            with open(audio_file_path, "rb") as audio_file:
                response = self.client.speech_to_text.transcribe(
                    file=audio_file,
                    model="saarika:v2.5",
                    language_code=language
                )

            if response and hasattr(response, 'transcript'):
                return response.transcript
            else:
                print("❌ STT: No transcript received")
                return ""

        except Exception as e:
            print(f"❌ STT Error: {str(e)}")
            return ""

# Initialize Sarvam AI
sarvam_ai = SarvamAI(os.getenv("SARVAM_API_KEY"))

class AIShoppingAgent:
    """Enhanced AI Agent for shopping assistance with Sarvam AI integration"""

    def __init__(self, memory, groq_client, collection, sarvam_ai):
        self.memory = memory
        self.groq_client = groq_client
        self.collection = collection
        self.sarvam_ai = sarvam_ai

    def get_system_prompt(self):
        cart_info = self.memory.get_cart_summary()
        current_step = self.memory.current_step
        conversation_context = self.memory.get_conversation_context(3)
        return f"""You are a helpful AI shopping assistant with a natural, conversational personality.

CURRENT SESSION INFO:
- Shopping Step: {current_step}
- {cart_info}

RECENT CONVERSATION:
{conversation_context}

YOUR CAPABILITIES:
1. **Product Discovery**: Help find products using the available inventory
2. **Cart Management**: Add, remove, update quantities
3. **Personalized Recommendations**: Based on user preferences and history
4. **Checkout Assistance**: Guide through purchase process

PERSONALITY GUIDELINES:
- Be friendly, helpful, and conversational
- Use emojis appropriately to make responses engaging
- Ask clarifying questions when needed
- Provide specific product details and recommendations
- Remember what we've discussed previously
- Keep responses concise but informative (max 2-3 sentences usually)"""

    def process_query(self, user_input, use_rag=True, enable_tts=False, tts_language="hi-IN", tts_speaker="Anushka"):
        """Process user query with optional TTS"""

        # Handle special commands
        if user_input.lower() in ["cart", "show cart", "view cart"]:
            response = self.memory.get_cart_summary()
        elif user_input.lower() in ["checkout", "buy now", "proceed"]:
            response = self.start_checkout()
        elif user_input.lower() in ["clear cart", "empty cart"]:
            self.memory.cart = []
            response = "🛒 Cart cleared! Ready to shop for new items."
        else:
            # Use RAG for product queries
            context = ""
            search_results = None
            if use_rag:
                try:
                    query_embedding = model.encode(user_input).tolist()
                    search_results = self.collection.query(
                        query_embeddings=[query_embedding],
                        n_results=3
                    )
                    context = "\n\n".join(search_results["documents"][0])
                except Exception as e:
                    print(f"RAG search error: {e}")
                    context = "No specific product context available."

            prompt = f"""{self.get_system_prompt()}

AVAILABLE PRODUCTS (if relevant):
{context}

USER INPUT: {user_input}

Instructions: Respond naturally and helpfully. If the user wants to add something to cart, be specific about which product."""

            try:
                completion = self.groq_client.chat.completions.create(
                    model="llama3-70b-8192",
                    messages=[
                        {"role": "system", "content": "You are a helpful, friendly AI shopping assistant."},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=0.7,
                    max_tokens=200
                )
                response = completion.choices[0].message.content.strip()
                self.handle_cart_operations(user_input, response, search_results)
            except Exception as e:
                response = f"❌ Sorry, I encountered an error: {str(e)[:100]}. Please try again."

        # Add to conversation history
        self.memory.add_conversation(user_input, response)

        # Generate TTS audio file if requested
        audio_file = None
        if enable_tts:
            try:
                audio_file = self.sarvam_ai.text_to_speech(response, tts_speaker, tts_language)
            except Exception as e:
                print(f"TTS Error: {e}")

        return response, audio_file

    def handle_cart_operations(self, user_input, ai_response, search_results):
        """Handle cart operations based on user input"""
        add_keywords = ["add to cart", "buy this", "purchase", "take this", "add it", "get this", "want this"]
        remove_keywords = ["remove", "delete", "take out", "don't want"]
        user_lower = user_input.lower()

        if any(keyword in user_lower for keyword in add_keywords) and search_results:
            if search_results["metadatas"][0]:
                product_metadata = search_results["metadatas"][0][0]
                self.memory.add_to_cart(product_metadata)
                self.memory.update_step("selecting")
        elif any(keyword in user_lower for keyword in remove_keywords):
            for item in self.memory.cart:
                if item["product"]["product_name"].lower() in user_lower:
                    self.memory.remove_from_cart(item["product"]["product_name"])
                    break

    def start_checkout(self):
        """Start checkout process"""
        if not self.memory.cart:
            return "🛒 Your cart is empty. Let's find some great products first! What are you looking for?"

        self.memory.update_step("checkout")
        checkout_msg = f"""🛍️ **Ready to Checkout!**

{self.memory.get_cart_summary()}

💳 **Checkout Process:**
1. 📍 Shipping address
2. 💸 Payment method
3. 📋 Order confirmation

Let's start! What's your shipping address?"""
        return checkout_msg

# Initialize the AI Shopping Agent with Sarvam AI
shopping_agent = AIShoppingAgent(shopping_memory, client, collection, sarvam_ai)

print("🛒 AI Shopping Assistant Initialized!")
print("✅ Sarvam AI SDK integration ready")
print("✅ TTS, STT services available")
print("✅ Memory and conversation tracking")
print("✅ Cart management")
print("\nReady for shopping interaction! 🎉")
print("\n📚 Sarvam AI Documentation: https://docs.sarvam.ai/api-reference-docs/")

🛒 AI Shopping Assistant Initialized!
✅ Sarvam AI SDK integration ready
✅ TTS, STT services available
✅ Memory and conversation tracking
✅ Cart management

Ready for shopping interaction! 🎉

📚 Sarvam AI Documentation: https://docs.sarvam.ai/api-reference-docs/


In [54]:
import gradio as gr
import tempfile
import os
from datetime import datetime

# Enhanced Gradio interface with ChatGPT-like flow and live cart updates

def chat_interface(message, history, enable_tts=False, tts_speaker="Anushka", tts_language="hi-IN"):
    """
    ChatGPT-like interface with conversation flow
    """
    if not message.strip():
        return history, "", shopping_memory.get_cart_summary(), None

    # Process the query
    response, audio_file = shopping_agent.process_query(
        message,
        use_rag=True,
        enable_tts=enable_tts,
        tts_language=tts_language,
        tts_speaker=tts_speaker
    )

    # Add to chat history
    history.append((message, response))

    return history, "", shopping_memory.get_cart_summary(), audio_file

def process_voice_input(audio_file, stt_language="hi-IN"):
    """Process voice input and return the transcript"""
    if audio_file is None:
        return "No audio file provided", []

    try:
        # Convert speech to text
        transcript = sarvam_ai.speech_to_text(audio_file, stt_language)
        if transcript:
            # Process the transcript as a chat message
            response, audio_response = shopping_agent.process_query(transcript, use_rag=True)

            # Create chat history entry
            chat_history = [(transcript, response)]

            return f"🎙️ Voice recognized: {transcript}", chat_history
        else:
            return "❌ Could not understand the audio", []
    except Exception as e:
        return f"Voice processing error: {str(e)}", []

def clear_cart():
    """Clear the shopping cart"""
    shopping_memory.cart = []
    return "🛒 Cart cleared! Ready to shop for new items."

def clear_chat():
    """Clear chat history"""
    shopping_memory.conversation_history = []
    return []

def generate_tts_demo(text, speaker="Anushka", language="hi-IN"):
    """Demo function for Sarvam TTS"""
    try:
        audio_file = sarvam_ai.text_to_speech(text, speaker, language)
        if audio_file:
            return f"🔊 Audio generated successfully!", audio_file
        else:
            return "❌ Failed to generate audio", None
    except Exception as e:
        return f"TTS error: {str(e)}", None

# Custom CSS for ChatGPT-like interface
custom_css = """
.gradio-container {
    font-family: 'Inter', sans-serif;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
}

.main-header {
    text-align: center;
    padding: 2rem;
    background: rgba(255, 255, 255, 0.1);
    border-radius: 15px;
    margin-bottom: 2rem;
    backdrop-filter: blur(10px);
}

.chat-container {
    background: rgba(255, 255, 255, 0.05);
    border-radius: 15px;
    padding: 1rem;
    backdrop-filter: blur(10px);
}

.cart-container {
    background: rgba(255, 255, 255, 0.1);
    border-radius: 15px;
    padding: 1rem;
    backdrop-filter: blur(10px);
    border: 2px solid rgba(255, 255, 255, 0.2);
}

.feature-box {
    background: rgba(255, 255, 255, 0.1);
    border-radius: 10px;
    padding: 1rem;
    margin: 0.5rem;
    backdrop-filter: blur(5px);
}

.voice-section {
    background: rgba(255, 255, 255, 0.08);
    border-radius: 12px;
    padding: 1.5rem;
    margin: 1rem 0;
    backdrop-filter: blur(8px);
}
"""

# Create the enhanced Gradio interface
with gr.Blocks(title="🛒 RRRR WALMART AI Shopping Assistant", theme=gr.themes.Soft(), css=custom_css) as demo:

    gr.HTML("""
    <div class="main-header">
        <h1 style="color: white; font-size: 3rem; margin-bottom: 1rem;">🛒 AI Shopping Assistant</h1>
        <h3 style="color: rgba(255,255,255,0.9); font-weight: 300;">
            Chat with AI • Voice Commands • Smart Cart Management
        </h3>
        <div style="display: flex; justify-content: center; gap: 20px; margin-top: 2rem;">
            <div class="feature-box">🔍 Smart Search</div>
            <div class="feature-box">🗣️ Voice Shopping</div>
            <div class="feature-box">🛒 Live Cart</div>
            <div class="feature-box">🔊 Audio Response</div>
        </div>
    </div>
    """)

    with gr.Tab("💬 Chat Shopping"):
        with gr.Row():
            # Chat Interface (Left Side)
            with gr.Column(scale=2, elem_classes="chat-container"):
                chatbot = gr.Chatbot(
                    label="🤖 AI Shopping Assistant",
                    height=500,
                    show_label=True,
                    bubble_full_width=False,
                    show_copy_button=True
                )

                with gr.Row():
                    msg_input = gr.Textbox(
                        placeholder="💬 Ask about products, add to cart, or say 'checkout'...",
                        label="Your Message",
                        lines=2,
                        container=True,
                        scale=4
                    )
                    send_btn = gr.Button("Send 📤", variant="primary", scale=1, size="lg")

                with gr.Row():
                    enable_tts = gr.Checkbox(label="🔊 Enable Voice Response", value=False)
                    tts_speaker = gr.Dropdown(
                        choices=["Anushka", "Manisha", "Vidya", "Arya", "Abhilash", "Karun", "Hitesh"],
                        value="Anushka",
                        label="🎤 Voice"
                    )
                    tts_language = gr.Dropdown(
                        choices=["hi-IN", "en-IN", "bn-IN", "ta-IN", "te-IN", "mr-IN", "gu-IN", "kn-IN"],
                        value="hi-IN",
                        label="🗣️ Language"
                    )

                # Audio output for TTS
                audio_output = gr.Audio(
                    label="🔊 Voice Response",
                    visible=True,
                    interactive=False,
                    type="filepath"
                )

                clear_chat_btn = gr.Button("🗑️ Clear Chat", variant="secondary", size="sm")

            # Cart & Actions (Right Side)
            with gr.Column(scale=1, elem_classes="cart-container"):
                cart_display = gr.Textbox(
                    label="🛒 Shopping Cart",
                    value=shopping_memory.get_cart_summary(),
                    lines=12,
                    interactive=False,
                    container=True
                )

                with gr.Row():
                    clear_cart_btn = gr.Button("🗑️ Clear Cart", variant="stop", size="lg")
                    checkout_btn = gr.Button("💳 Checkout", variant="primary", size="lg")

    with gr.Tab("🎙️ Voice Shopping"):
        gr.HTML('<div class="voice-section">')
        gr.Markdown("### 🗣️ Shop with Your Voice!")

        with gr.Row():
            with gr.Column():
                voice_input = gr.Audio(
                    label="🎤 Record or Upload Audio",
                    type="filepath",
                    interactive=True
                )

                stt_language = gr.Dropdown(
                    choices=["hi-IN", "en-IN", "bn-IN", "ta-IN", "te-IN", "mr-IN", "gu-IN", "kn-IN"],
                    value="hi-IN",
                    label="🗣️ Audio Language"
                )

                process_voice_btn = gr.Button("🎙️ Process Voice Command", variant="primary", size="lg")

            with gr.Column():
                voice_result = gr.Textbox(
                    label="🎙️ Voice Recognition Result",
                    lines=4,
                    interactive=False
                )

                voice_chat = gr.Chatbot(
                    label="Voice Chat Result",
                    height=300,
                    show_label=True
                )

        gr.HTML('</div>')

        gr.Markdown("### 📝 Voice Commands Examples:")
        gr.Markdown("""
        - **English**: "Show me fruits under two dollars"
        - **Hindi**: "मुझे दो डॉलर से कम फल दिखाओ"
        - **Bengali**: "দুই ডলারের নিচে ফল দেখান"
        """)

    with gr.Tab("🔊 Text-to-Speech"):
        gr.Markdown("### 🔊 Convert Text to Speech")

        with gr.Row():
            with gr.Column():
                tts_text = gr.Textbox(
                    lines=3,
                    placeholder="Enter text to convert to speech...",
                    label="Text Input"
                )

                with gr.Row():
                    tts_demo_speaker = gr.Dropdown(
                        choices=["Anushka", "Manisha", "Vidya", "Arya", "Abhilash", "Karun", "Hitesh"],
                        value="Anushka",
                        label="🎤 Voice"
                    )
                    tts_demo_language = gr.Dropdown(
                        choices=["hi-IN", "en-IN", "bn-IN", "ta-IN", "te-IN", "mr-IN", "gu-IN", "kn-IN"],
                        value="hi-IN",
                        label="🗣️ Language"
                    )

                tts_btn = gr.Button("🔊 Generate Speech", variant="primary", size="lg")

            with gr.Column():
                tts_status = gr.Textbox(
                    lines=4,
                    label="TTS Status",
                    interactive=False
                )

                tts_audio_demo = gr.Audio(
                    label="🔊 Generated Audio",
                    interactive=False,
                    type="filepath"
                )

    # Event handlers with proper state management

    # Main chat interface
    send_btn.click(
        fn=chat_interface,
        inputs=[msg_input, chatbot, enable_tts, tts_speaker, tts_language],
        outputs=[chatbot, msg_input, cart_display, audio_output]
    )

    msg_input.submit(
        fn=chat_interface,
        inputs=[msg_input, chatbot, enable_tts, tts_speaker, tts_language],
        outputs=[chatbot, msg_input, cart_display, audio_output]
    )

    # Voice processing
    process_voice_btn.click(
        fn=process_voice_input,
        inputs=[voice_input, stt_language],
        outputs=[voice_result, voice_chat]
    )

    # Cart management
    clear_cart_btn.click(
        fn=clear_cart,
        outputs=cart_display
    )

    checkout_btn.click(
        fn=lambda: shopping_agent.start_checkout(),
        outputs=cart_display
    )

    # Chat management
    clear_chat_btn.click(
        fn=clear_chat,
        outputs=chatbot
    )

    # TTS demo
    tts_btn.click(
        fn=generate_tts_demo,
        inputs=[tts_text, tts_demo_speaker, tts_demo_language],
        outputs=[tts_status, tts_audio_demo]
    )

    # Auto-update cart after voice commands
    process_voice_btn.click(
        fn=lambda: shopping_memory.get_cart_summary(),
        outputs=cart_display
    )

# Launch the interface
print("🚀 Launching enhanced RRRR WALMART AI Shopping Assistant...")
demo.launch(
    share=True,
    server_name="0.0.0.0",
    server_port=9003,
    show_error=True,
    inbrowser=True
)

/tmp/ipython-input-54-252527258.py:141: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-54-252527258.py:141: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(
/tmp/ipython-input-54-252527258.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  voice_chat = gr.Chatbot(


🚀 Launching enhanced Sarvam AI Shopping Assistant...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://beff5683fc1bb81471.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [55]:
# Quick test to ensure everything works
print("🔧 Testing Gradio compatibility...")

# Test basic shopping query
test_response = shopping_agent.process_query("Show me dairy products", use_rag=True)
print(f"Test query response: {test_response[:100]}...")

print("✅ All systems ready!")
print("🚀 Launch the interface by running the next cell!")

🔧 Testing Gradio compatibility...
Test query response: ("🛍️ Hi there! I'd be happy to show you our dairy products! 🥛 We have a few options from Great Value that caught my eye. We have a gallon of Skim Milk for $3.78, a half-gallon of Organic Whole Vitamin D Milk for $4.48, and a gallon of 2% Reduced Fat Milk for $3.88. Which one are you interested in? 🤔", None)...
✅ All systems ready!
🚀 Launch the interface by running the next cell!


In [56]:
# 🧪 Test Enhanced AI Shopping Assistant

print("🧪 Testing Enhanced AI Shopping Assistant...")
print("=" * 60)

# Test 1: Text-to-Speech
print("\n🔊 Testing Text-to-Speech Service:")
try:
    tts_result = sarvam_ai.text_to_speech("नमस्ते! AI शॉपिंग असिस्टेंट में आपका स्वागत है।", "Anushka", "hi-IN")
    if tts_result:
        print(f"✅ TTS successful: {tts_result}")
    else:
        print("❌ TTS failed - check API key and connection")
except Exception as e:
    print(f"TTS error: {e}")

# Test 2: Shopping Assistant
print("\n🛍️ Testing Shopping Assistant:")
test_query = "Show me fruits under $2"
try:
    response, audio_file = shopping_agent.process_query(test_query, enable_tts=False)
    print(f"Query: {test_query}")
    print(f"Response: {response}")
    if audio_file:
        print(f"Audio generated: {audio_file}")
except Exception as e:
    print(f"Shopping assistant error: {e}")

# Test 3: Cart Management
print("\n🛒 Testing Cart Management:")
print(f"Current cart: {shopping_memory.get_cart_summary()}")

print("\n✅ Enhanced AI Shopping Assistant Test Complete!")
print("\n📚 New Features:")
print("• ChatGPT-like conversation flow")
print("• Working Text-to-Speech with audio playback")
print("• Speech-to-Text for voice commands")
print("• Live cart updates")
print("• Removed translation features")
print("• Enhanced UI with better user engagement")
print("\n🚀 Ready to launch the new interface!")

🧪 Testing Enhanced AI Shopping Assistant...

🔊 Testing Text-to-Speech Service:
❌ TTS Error: headers: {'date': 'Sun, 13 Jul 2025 17:59:40 GMT', 'content-type': 'application/json', 'content-length': '347', 'connection': 'keep-alive', 'server': 'uvicorn', 'x-request-id': '20250713_47695efe-5f70-4b3a-827c-c55e1bfe69d4', 'access-control-allow-origin': '*', 'strict-transport-security': 'max-age=31536000; includeSubDomains; preload'}, status_code: 400, body: {'error': {'message': "Validation Error(s):\n- speaker: Input should be 'meera', 'pavithra', 'maitreyi', 'arvind', 'amol', 'amartya', 'diya', 'neel', 'misha', 'vian', 'arjun', 'maya', 'anushka', 'abhilash', 'manisha', 'vidya', 'arya', 'karun' or 'hitesh'", 'code': 'invalid_request_error', 'request_id': '20250713_47695efe-5f70-4b3a-827c-c55e1bfe69d4'}}
❌ TTS failed - check API key and connection

🛍️ Testing Shopping Assistant:
Query: Show me fruits under $2
Response: 🍉 Ah, fruits under $2, eh? 🤔 I've got just the thing for you! 😊 We have s

# 🚀 Launch Your AI Shopping Assistant


Your complete AI shopping assistant is now ready! Here's what you can do:


## Features:
- 💬 **Text Chat**: Type your queries for quick responses  
- 🧠 **Smart Memory**: Remembers your preferences and cart items
- 🔍 **Product Search**: Uses RAG to find relevant products
- 🛒 **Cart Management**: Add, view, and manage items
- 💳 **Checkout Process**: Complete guided purchase flow


## How to Use:
1. **Run all cells above** to initialize the system
2. **Execute the cell below** to start shopping
3. **Interact with the Gradio interface**
4. **Shop, add to cart, and checkout!**


## Example Commands:
- "Show me fruits under $2"
- "Add the best apple to my cart"
- "What's in my cart?"
- "I'm ready to checkout"


Ready to start shopping? Run the cell below! 👇


In [57]:
# 🎉 LAUNCH YOUR ENHANCED AI SHOPPING ASSISTANT!

print("🛒 Enhanced AI Shopping Assistant Ready!")
print("=" * 70)
print()
print("🚀 **NEW CHATGPT-LIKE FEATURES:**")
print("• 💬 Real-time chat interface with conversation flow")
print("• 🔊 Working Text-to-Speech with audio playback")
print("• 🎙️ Speech-to-Text voice commands")
print("• 🛒 Live cart updates that sync automatically")
print("• 🎨 Beautiful modern UI with smooth interactions")
print("• ⚡ Fast response times and engaging user experience")
print()
print("📋 **COMPLETE SYSTEM INCLUDES:**")
print("✅ ChatGPT-style conversation interface")
print("✅ Working voice input and output")
print("✅ RAG-powered intelligent product search")
print("✅ Real-time shopping cart management")
print("✅ Conversational AI with memory")
print("✅ Official Sarvam AI SDK integration")
print("✅ Multi-language voice support")
print()
print("🎯 **HOW TO USE:**")
print("1. 💬 Chat Tab: Type messages for instant responses")
print("2. 🎙️ Voice Tab: Record or upload audio for voice shopping")
print("3. 🔊 TTS Tab: Test text-to-speech functionality")
print("4. 🛒 Cart updates automatically with each interaction")
print("5. ✨ Enable voice response for audio feedback")
print()
print("🌟 **SAMPLE COMMANDS:**")
print("• 'Show me fruits under $2'")
print("• 'Add apple to my cart'")
print("• 'What's in my cart?'")
print("• 'I want to checkout'")
print()
print("📚 **Sarvam AI Documentation:**")
print("https://docs.sarvam.ai/api-reference-docs/")
print()
print("🎊 **Experience the future of AI-powered shopping!**")
print("🚀 Your ChatGPT-like shopping assistant is launching...")

🛒 Enhanced AI Shopping Assistant Ready!

🚀 **NEW CHATGPT-LIKE FEATURES:**
• 💬 Real-time chat interface with conversation flow
• 🔊 Working Text-to-Speech with audio playback
• 🎙️ Speech-to-Text voice commands
• 🛒 Live cart updates that sync automatically
• 🎨 Beautiful modern UI with smooth interactions
• ⚡ Fast response times and engaging user experience

📋 **COMPLETE SYSTEM INCLUDES:**
✅ ChatGPT-style conversation interface
✅ Working voice input and output
✅ RAG-powered intelligent product search
✅ Real-time shopping cart management
✅ Conversational AI with memory
✅ Official Sarvam AI SDK integration
✅ Multi-language voice support

🎯 **HOW TO USE:**
1. 💬 Chat Tab: Type messages for instant responses
2. 🎙️ Voice Tab: Record or upload audio for voice shopping
3. 🔊 TTS Tab: Test text-to-speech functionality
4. 🛒 Cart updates automatically with each interaction
5. ✨ Enable voice response for audio feedback

🌟 **SAMPLE COMMANDS:**
• 'Show me fruits under $2'
• 'Add apple to my cart'
• 'What